In [16]:
import pandas as pd
import requests as rq
import json
from datetime import datetime as dt
import ast

In [17]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [18]:
market = json.loads(api.text)

In [19]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [20]:
roundN = pd.DataFrame(market["atletas"])

In [21]:
with open("../../status_id.json") as file:
    status_id = json.load(file)
with open("../../positions.json") as file:
    pos_id = json.load(file)

In [ ]:
roundN = roundN[["rodada_id", "atleta_id", "nome", "apelido", "posicao_id", "clube_id", "scout", "pontos_num", "preco_num", "variacao_num"]]
roundN.columns = ["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "scout", "total_score", "current_price", "price_delta"]

In [22]:
roundN.position = roundN.position.apply(lambda x: pos_id[str(x)])

In [ ]:
def scout_parser(scout, col_names):
    scout_dict = ast.literal_eval(scout)
    scout_list = []
    for col in col_names:
        try:
            el = scout_dict[col]
        except:
            el = 0
        scout_list.append(el)
    return pd.DataFrame(columns = col_names, data = [scout_list]).iloc[0]

In [ ]:
col_names = ["G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC"]
roundN[col_names] = roundN.scout.apply(lambda x: scout_parser(x, col_names))
roundN = roundN[["round_num", "player_id", "player_name", "player_nickname", "position", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price", "price_delta"]]

In [23]:
roundN.head(10)

,apelido,atleta_id,clube_id,foto,jogos_num,media_num,nome,pontos_num,posicao_id,preco_num,rodada_id,scout,slug,status_id,variacao_num
0,Renê,78445,262,https://s.glbimg.com/es/sde/f/2019/04/26/f7f5e...,4,3.27,Renê Rodrigues Martins,0,LAT,7.72,7,None,rene,7,0.00
1,Bruno Henrique,90285,262,https://s.glbimg.com/es/sde/f/2019/04/26/195ab...,6,6.80,Bruno Henrique Pinto,0,ATA,20.56,7,None,bruno-henrique,7,0.00
2,Fagner,42500,264,https://s.glbimg.com/es/sde/f/2019/03/18/d5cd8...,5,9.90,Fagner Conserva Lemos,0,LAT,15.29,7,None,fagner,6,0.00
3,Dedé,60819,283,https://s.glbimg.com/es/sde/f/2018/05/18/868bc...,7,5.72,Anderson Vital da Silva,0,ZAG,15.74,7,None,dede,7,0.14
4,Fábio Santos,38229,282,https://s.glbimg.com/es/sde/f/2018/05/07/98bee...,6,4.27,Fábio Santos Romeu,0,LAT,12.61,7,None,fabio-santos,7,3.09
5,Fábio,37656,283,https://s.glbimg.com/es/sde/f/2018/05/18/d4072...,6,3.17,Fábio Deivson Lopes Maciel,0,GOL,9.64,7,None,fabio,7,-0.46
6,Nico López,84709,285,https://s.glbimg.com/es/sde/f/2018/04/08/5bb6e...,6,3.32,Nicolás Federico López Alonso,0,ATA,15.14,7,None,nico-lopez,7,-0.49
7,Gabriel,83257,262,https://s.glbimg.com/es/sde/f/2019/04/26/dd351...,5,7.74,Gabriel Barbosa Almeida,0,ATA,18.33,7,None,gabriel,7,2.61
8,Diego Alves,38509,262,https://s.glbimg.com/es/sde/f/2019/04/26/7bdef...,4,2.50,Diego Alves Carreira,0,GOL,5.26,7,None,diego-alves,7,1.06
9,Everton,86757,284,https://s.glbimg.com/es/sde/f/2018/05/18/997fe...,6,6.68,Everton Sousa Soares,0,ATA,17.31,7,None,everton,6,0.00


In [7]:
roundN.to_csv("2019_round07_status.csv")

In [8]:
status = roundN[["rodada_id", "atleta_id", "status_id"]]

In [24]:
status.rodada_id = status.rodada_id + 1
now = dt.now()
status["query_time"] = dt(now.year, now.month, now.day, now.hour, now.minute, now.second)
status.status_id = status.status_id.apply(lambda x: status_id[str(x)])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
status = status[["rodada_id", "query_time", "atleta_id", "status_id"]]
status.columns = ["round_num", "query_time", "player_id", "status_id"]
status.head()

In [28]:
filename = "../status/2019_round"
if status.round_num[0] < 10:
    filename = filename + "0" + str(status.round_num[0]) + "_status.csv"
else:
    filename = filename + str(status.round_num[0]) + "_status.csv"

status.to_csv(filename, index=False)